In [58]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append('./../../../../')
sys.path.append('./../../../script/')
sys.path.append('./')
sys.path.append('./designer/script/')

from tmm.get_E import get_E_free
from tmm.get_jacobi_arb_adjoint import get_jacobi_E_free_form
from tmm.get_jacobi_adjoint import get_jacobi_adjoint
from film import FreeFormFilm

import numpy as np



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Manual differentiation

In [4]:

wls = np.linspace(500, 1000, 3)
inc_ang = 0.
np.random.seed(0)
film = FreeFormFilm(np.random.random(4) * 1 + 1.5, 1000, 1)
jacobian = np.zeros((wls.shape[0] * 4, film.get_d().shape[0], 2, 2), dtype='complex128')

get_jacobi_E_free_form(
    jacobian, wls, film.get_d(), 
    film.calculate_n_array(wls), 
    film.calculate_n_sub(wls), 
    film.calculate_n_inc(wls), 
    inc_ang, 
)

E = np.zeros((wls.shape[0] * 2, 2), dtype='complex128')
get_E_free(E, wls, film.get_d(), 
    film.calculate_n_array(wls), 
    film.calculate_n_sub(wls), 
    film.calculate_n_inc(wls), 
    inc_ang, 
)


jacobian_gt = np.zeros((wls.shape[0] * 2, film.get_d().shape[0]))
get_jacobi_adjoint(
    jacobian_gt, wls, film.get_d(), 
    film.calculate_n_array(wls), 
    film.calculate_n_sub(wls), 
    film.calculate_n_inc(wls), 
    inc_ang, 
)


/home/vipuser/miniconda3/envs/tf_design/lib/python3.10/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/vipuser/miniconda3/envs/tf_design/lib/python3.10/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [59]:
# autograd
from tmm.autograd_wrapper import *
from tmm.E_to_spectrum import E_to_R, E_to_tan2Psi

# set exp 
wls = np.linspace(500, 1000, 500)
film = FreeFormFilm(np.random.rand(20) * 1 + 1.5, 1000, 2)
d = film.get_d()
n_layers, n_sub, n_inc = film.calculate_n_array(wls), film.calculate_n_sub(wls), film.calculate_n_inc(wls)


# objective: 0.5
def loss_fn(E):
    e = E_to_R(E)
    dif = e - (torch.ones_like(e) - 0.5)
    return dif.square().sum()

jacobi_auto = get_jacobi_warpper(loss_fn)
jacobi = np.zeros((wls.shape[0] * 4, d.shape[0], 2, 2))


jacobi = jacobi_auto(    
    jacobi,
    wls,
    d,
    n_layers,
    n_sub,
    n_inc,
    inc_ang,    
)



RuntimeError: Index put requires the source and destination dtypes match, got Double for the destination and ComplexDouble for the source.